In [1]:
# Carga de variables de entorno
from dotenv import load_dotenv

# LangChain y Google Generative AI
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.docstore.document import Document as LangchainDocument

# Visualización en Jupyter
from IPython.display import display, Markdown

# Manipulación de datos
import pandas as pd
import numpy as np
import pickle

# Manejo de archivos y peticiones web
import os
import requests
import zipfile
import json

# Procesamiento XML y HTML
import lxml
import lxml.etree as ET
from bs4 import BeautifulSoup

# Expresiones regulares
import re

# Procesamiento de lenguaje natural
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import language_tool_python

# LangGraph
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Descargar recursos necesarios de NLTK (se recomienda ejecutar al inicio del notebook)
nltk.download('punkt')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')

[nltk_data] Downloading package punkt to C:\Users\Luis
[nltk_data]     Carreras\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package perluniprops to C:\Users\Luis
[nltk_data]     Carreras\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to C:\Users\Luis
[nltk_data]     Carreras\AppData\Roaming\nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


True

# **Extracción de la información de medicamentos**



Descargamos *medicamentos.xls*de la web de la aemps.

In [2]:
URL = "https://listadomedicamentos.aemps.gob.es/medicamentos.xls"
response = requests.get(URL)
open("medicamentos.xls", "wb").write(response.content)

2724369

Convertimos la información de los medicamenots en un dataframe y  añadimos la columna *nombre* a este dataframe con la primera palabra del nombre comercial

In [3]:
medicamentos=pd.read_excel('medicamentos.xls')

# Extraemos la primera palabra de Medicamento como nombre abreviado
medicamentos['Nombre']=medicamentos['Medicamento'].str.split(' ',expand=True)[0]
medicamentos.to_csv('medicamentos.csv')

#A los que empiezan por "ácido" se les pone sus dos primeras palabras (Ácido acetilsalicílico) como nombre abreviado
miniMed = medicamentos[medicamentos['Nombre'].str.lower()=='acido']
miniMed['Nombre'] = miniMed['Medicamento'].str.split().apply(lambda x: ' '.join(x[:2]))


merged = medicamentos.merge(miniMed, on='Medicamento', how='left', suffixes=('', '_y'))
medicamentos['Nombre'] = merged['Nombre_y'].combine_first(medicamentos['Nombre']) # Si Nombre_y NO es nulo (NaN) se usa para reemplazar el valor de Nombre. Si Nombre_y ES nulo (NaN) se deja el valor original de Nombre.

medicamentos.to_csv('medicamentos.csv')

C:\Users\Luis Carreras\AppData\Local\Temp\ipykernel_20548\93027074.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miniMed['Nombre'] = miniMed['Medicamento'].str.split().apply(lambda x: ' '.join(x[:2]))


Guardamos unos pocos, vamos a trabajar con estos

In [4]:
medicamentos_prueba=medicamentos[0:10]
medicamentos_prueba

,Nº Registro,Medicamento,Laboratorio,Fecha Aut.,Estado,Fecha Estado,Cód. ATC,Principios Activos,Nº P. Activos,¿Comercializado?,¿Triangulo Amarillo?,Observaciones,¿Sustituible?,¿Afecta conducción?,¿Problemas de suministro?,Nombre
0,06354005IP1,COMPETACT 15 MG/850 MG COMPRIMIDOS RECUBIERTOS...,Takeda Pharma A/S,19/05/2022,Autorizado,20/05/2022,A10BD05,"METFORMINA HIDROCLORURO, PIOGLITAZONA HIDROCLO...",2,NO,NO,Medicamento Sujeto A Prescripción Médica,NaN,NO,NO,COMPETACT
1,06356001,EXJADE 125 MG COMPRIMIDOS DISPERSABLES,Novartis Europharm Limited,09/10/2006,Anulado,08/02/2022,V03AC03,DEFERASIROX,1,NO,NO,Diagnóstico Hospitalario,NaN,NO,NO,EXJADE
2,06361001,LUMINITY 150 MICROLITROS/ML GAS Y DISOLVENTE P...,Lantheus Eu Limited,09/10/2006,Autorizado,23/08/2017,V08DA04,PERFLUTRENO,1,SI,NO,Uso Hospitalario,NaN,NO,NO,LUMINITY
3,06363006,SPRYCEL 70 MG COMPRIMIDOS RECUBIERTOS CON PELI...,Bristol-Myers Squibb Pharma Eeig,29/11/2006,Autorizado,29/11/2006,L01EA02,DASATINIB MONOHIDRATO,1,SI,NO,Diagnóstico Hospitalario,NaN,SI,NO,SPRYCEL
4,06363015,SPRYCEL 140 MG COMPRIMIDOS RECUBIERTOS CON PEL...,Bristol-Myers Squibb Pharma Eeig,23/08/2011,Autorizado,23/08/2011,L01EA02,DASATINIB MONOHIDRATO,1,NO,NO,Diagnóstico Hospitalario,NaN,SI,NO,SPRYCEL
5,06364007IP,ADROVANCE 70 MG/5.600 UI COMPRIMIDOS,Organon N.V.,12/07/2024,Autorizado,13/07/2024,M05BB03,"COLECALCIFEROL, ALENDRONATO SODIO TRIHIDRATO",2,SI,NO,Medicamento Sujeto A Prescripción Médica,NaN,NO,NO,ADROVANCE
6,06366018,TANDEMACT 30 mg/2 mg COMPRIMIDOS,Cheplapharm Arzneimittel Gmbh,29/09/2009,Autorizado,29/09/2009,A10BD06,"PIOGLITAZONA, GLIMEPIRIDA",2,SI,NO,Medicamento Sujeto A Prescripción Médica,NaN,SI,NO,TANDEMACT
7,06367002,DIACOMIT 250 mg CAPSULAS DURAS,Biocodex,12/11/2009,Autorizado,12/11/2009,N03AX17,ESTIRIPENTOL,1,SI,NO,Diagnóstico Hospitalario,NaN,NO,NO,DIACOMIT
8,06367008,DIACOMIT 250 mg POLVO PARA SUSPENSION ORAL,Biocodex,12/11/2009,Autorizado,12/11/2009,N03AX17,ESTIRIPENTOL,1,SI,NO,Diagnóstico Hospitalario,NaN,NO,NO,DIACOMIT
9,06380003,PREZISTA 400 mg COMPRIMIDOS RECUBIERTOS CON PE...,Janssen-Cilag International N.V,16/02/2009,Autorizado,16/02/2009,J05AE10,DARUNAVIR ETANOLATO,1,SI,NO,Uso Hospitalario,NaN,NO,NO,PREZISTA


### Añadimos la información del prospecto al dataframe

Página para acceder a los prospectos. Necesitamos el número de registro del medicamento, el cual se encuentra en el dataframe.

In [5]:

#codigo='06367002' #Diacomit cápsulas duras
codigo='06367008' #Diacomit polvo para solución oral
url='https://cima.aemps.es/cima/dochtml/ft/'+codigo+'/FichaTecnica.html'
resp = requests.get(url)
print(url)

https://cima.aemps.es/cima/dochtml/ft/06367008/FichaTecnica.html


Se extraen los medicamentos del data_frame que no tienen información del prospecto.

In [6]:

# Función para obtener el HTML de la página de la ficha técnica
def obtener_ficha_tecnica(codigo):
    url = f'https://cima.aemps.es/cima/dochtml/ft/{codigo}/FichaTecnica.html'
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.text
    else:
        print(f"Error al obtener la página para el código {codigo}: {resp.status_code}")
        return None

# Función para extraer la información de la ficha técnica
def extraer_info_ficha_TF(codigo):
    ficha_html = obtener_ficha_tecnica(codigo)
    if ficha_html:
        return True  # Si la página se obtuvo correctamente
    else:
        return False  # Si ocurrió un error 404 o no se pudo obtener la página

# Recorrer todos los medicamentos y eliminar los que den error 404
medicamentos_validos = []

for index, row in medicamentos_prueba.iterrows():
    codigo = str(row['Nº Registro'])  
    if extraer_info_ficha_TF(codigo):  # Si la ficha técnica está disponible
        medicamentos_validos.append(row)  # Añadimos el medicamento a la lista de válidos
    else:
        print(f"Eliminado el medicamento con código {codigo} debido a error 404")

# Crear un nuevo DataFrame con los medicamentos válidos
medicamentos_filtrados = pd.DataFrame(medicamentos_validos)

Error al obtener la página para el código 06354005IP1: 404
Eliminado el medicamento con código 06354005IP1 debido a error 404
Error al obtener la página para el código 06364007IP: 404
Eliminado el medicamento con código 06364007IP debido a error 404


Dividimos la información del prospecto en secciones, añadimos esta información información en la nueva columna 'Prospecto'.

In [7]:

# Función para extraer la información de la ficha técnica preservando la estructura.
def extraer_info_ficha_bien(codigo):
    ficha_html = obtener_ficha_tecnica(codigo)
    if ficha_html:
        soup = BeautifulSoup(ficha_html, 'html.parser')
        
        # Si hay un título en <h1> se extrae, aunque puede que no lo necesites para separar secciones.
        titulo = soup.find('h1')
        if titulo:
            titulo = titulo.text.strip()
        else:
            titulo = "No se encontró título"
        
        # Extraer todo el texto preservando saltos de línea
        descripcion = soup.get_text(separator="\n")
        
        # Limpiar frases no útiles.
        descripcion = descripcion.replace("pulse aquí para ver el documento en formato PDF", "").strip()
        
        if not descripcion:
            descripcion = "No se encontró descripción"

        return descripcion
    else:
        return None

# Función para extraer secciones usando expresiones regulares
def extraer_secciones(texto):
    # Dividir el texto en líneas
    lineas = texto.splitlines()
    
    secciones = {}
    titulo_seccion = None
    contenido = []

    patron_encabezado = re.compile(r'^(\d+(\.\d+)*\.\s+.*|[A-Z\sÁÉÍÓÚÑ]+)$')
    # re.compile(r'^(\d+(\.\d+)*\.\s+.*|[A-Z\sÁÉÍÓÚÑ]+)$') un patrón de encabezado:
    # - Líneas que comiencen con dígitos y un punto (por ejemplo, "1. NOMBRE...", "4.1. Indicaciones...")
    # - O líneas que estén en mayúsculas y puedan corresponder a títulos (como "ADVERTENCIA TRIÁNGULO NEGRO")


    for linea in lineas:
        linea = linea.strip()
        if not linea:
            continue  # saltar líneas vacías

        if patron_encabezado.match(linea):
            # Si ya hay un título de sección, guardamos su contenido
            if titulo_seccion:
                secciones[titulo_seccion] = "\n".join(contenido).strip()
            titulo_seccion = linea
            contenido = []
        else:
            contenido.append(linea)
    
    # Guardar la última sección. Cuando termina el bucle, aún queda texto de la última sección que no se había guardado.
    if titulo_seccion:
        secciones[titulo_seccion] = "\n".join(contenido).strip()
    else:
        # Si no se detectaron encabezados, podemos devolver todo el texto en una sola sección.
        secciones["Texto Completo"] = texto

    return secciones

# Lista para almacenar las descripciones de cada medicamento
prospectos = []

# Recorremos el DataFrame fila por fila
for index, row in medicamentos_filtrados.iterrows():
    codigo = str(row['Nº Registro'])  # Convertimos el código a cadena
    descripcion = extraer_info_ficha_bien(codigo)

    if descripcion:
        secciones = extraer_secciones(descripcion)
        prospecto_texto = ""

        for encabezado, contenido in secciones.items():
            prospecto_texto += f"Encabezado: {encabezado}\n"
            prospecto_texto += f"Contenido: {contenido}\n"
            prospecto_texto += "-" * 40 + "\n"

        # Eliminar la sección 1. NOMBRE DEL MEDICAMENTO si está presente
        prospecto_texto = re.sub(
            r"Encabezado: 1\. NOMBRE DEL MEDICAMENTO\nContenido:.*?\n-{40}\n", 
            "", 
            prospecto_texto, 
            flags=re.DOTALL
        )

        # Añadimos el texto limpio
        prospectos.append(prospecto_texto.strip())
    else:
        prospectos.append("No se pudo extraer la ficha técnica.")

# Añadir columna al DataFrame
medicamentos_filtrados['Prospecto'] = prospectos

# Mostramos el DataFrame actualizado
medicamentos_filtrados


,Nº Registro,Medicamento,Laboratorio,Fecha Aut.,Estado,Fecha Estado,Cód. ATC,Principios Activos,Nº P. Activos,¿Comercializado?,¿Triangulo Amarillo?,Observaciones,¿Sustituible?,¿Afecta conducción?,¿Problemas de suministro?,Nombre,Prospecto
1,06356001,EXJADE 125 MG COMPRIMIDOS DISPERSABLES,Novartis Europharm Limited,09/10/2006,Anulado,08/02/2022,V03AC03,DEFERASIROX,1,NO,NO,Diagnóstico Hospitalario,NaN,NO,NO,EXJADE,Encabezado: ADVERTENCIA TRIÁNGULO NEGRO\nConte...
2,06361001,LUMINITY 150 MICROLITROS/ML GAS Y DISOLVENTE P...,Lantheus Eu Limited,09/10/2006,Autorizado,23/08/2017,V08DA04,PERFLUTRENO,1,SI,NO,Uso Hospitalario,NaN,NO,NO,LUMINITY,Encabezado: 2. COMPOSICIÓN CUALITATIVA Y CUANT...
3,06363006,SPRYCEL 70 MG COMPRIMIDOS RECUBIERTOS CON PELI...,Bristol-Myers Squibb Pharma Eeig,29/11/2006,Autorizado,29/11/2006,L01EA02,DASATINIB MONOHIDRATO,1,SI,NO,Diagnóstico Hospitalario,NaN,SI,NO,SPRYCEL,Encabezado: SPRYCEL\nContenido: 140\nmg compri...
4,06363015,SPRYCEL 140 MG COMPRIMIDOS RECUBIERTOS CON PEL...,Bristol-Myers Squibb Pharma Eeig,23/08/2011,Autorizado,23/08/2011,L01EA02,DASATINIB MONOHIDRATO,1,NO,NO,Diagnóstico Hospitalario,NaN,SI,NO,SPRYCEL,Encabezado: SPRYCEL\nContenido: 140\nmg compri...
6,06366018,TANDEMACT 30 mg/2 mg COMPRIMIDOS,Cheplapharm Arzneimittel Gmbh,29/09/2009,Autorizado,29/09/2009,A10BD06,"PIOGLITAZONA, GLIMEPIRIDA",2,SI,NO,Medicamento Sujeto A Prescripción Médica,NaN,SI,NO,TANDEMACT,Encabezado: 2. COMPOSICIÓN CUALITATIVA Y CUANT...
7,06367002,DIACOMIT 250 mg CAPSULAS DURAS,Biocodex,12/11/2009,Autorizado,12/11/2009,N03AX17,ESTIRIPENTOL,1,SI,NO,Diagnóstico Hospitalario,NaN,NO,NO,DIACOMIT,Encabezado: 2. COMPOSICIÓN CUALITATIVA Y CUANT...
8,06367008,DIACOMIT 250 mg POLVO PARA SUSPENSION ORAL,Biocodex,12/11/2009,Autorizado,12/11/2009,N03AX17,ESTIRIPENTOL,1,SI,NO,Diagnóstico Hospitalario,NaN,NO,NO,DIACOMIT,Encabezado: 2. COMPOSICIÓN CUALITATIVA Y CUANT...
9,06380003,PREZISTA 400 mg COMPRIMIDOS RECUBIERTOS CON PE...,Janssen-Cilag International N.V,16/02/2009,Autorizado,16/02/2009,J05AE10,DARUNAVIR ETANOLATO,1,SI,NO,Uso Hospitalario,NaN,NO,NO,PREZISTA,Encabezado: 2. COMPOSICIÓN CUALITATIVA Y CUANT...


# **Creación del modelo de lenguaje**

Llamamos a nuestro modelo de lenguaje.

In [8]:
load_dotenv()
modelname = "gemini-2.5-flash-preview-04-17"
llm = ChatGoogleGenerativeAI(model=modelname,temperature=0, max_tokens=50000)


Les aplicamos embeddings a los prospectos de los medicamentos y los almacenamos en un vector store.

In [9]:

# Cargar variables de entorno
load_dotenv()

# Inicializar modelos
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0, max_tokens=50000)


# Ruta para guardar o cargar el índice
faiss_path = "faiss_index"
metadata_path = "documentos_metadata.pkl"


# Cargamos y si no esta cargado creamos el vector store (FAISS).

if os.path.exists(faiss_path):
    print("Cargando vector store desde disco...")
    vectordb = FAISS.load_local(faiss_path, embedding_model, allow_dangerous_deserialization=True)

    with open(metadata_path, "rb") as f:
        documentos = pickle.load(f)

else:
    print("Calculando embeddings y creando vector store...")
    documentos = []

    for _, row in medicamentos_filtrados.iterrows():
        texto = row['Prospecto']
        metadata = row.to_dict()

        try:
            doc = LangchainDocument(page_content=texto, metadata=metadata)
            documentos.append(doc)
        except Exception as e:
            print(f"Error con el medicamento {row['Nombre']}: {e}")

    vectordb = FAISS.from_documents(documentos, embedding_model)
    vectordb.save_local(faiss_path)

    with open(metadata_path, "wb") as f:
        pickle.dump(documentos, f)

print("Vector store listo.")

Cargando vector store desde disco...
Vector store listo.


Creamos nuestro modelo proporcionando la información relevante mediante similitud de cosenos.

In [10]:

def calcular_embeddings(query, embedding_model):
    """
    Calcula el embedding de la consulta.
    """
    return np.array(embedding_model.embed_query(query)).reshape(1, -1)

def obtener_vectores_faiss(vectordb):
    """
    Extrae los vectores desde el índice FAISS. 
    """
    # Accedemos todos los vectores que están almacenados en el índice FAISS. FAISS almacena los vectores internamente.
    return vectordb.index.reconstruct_n(0, vectordb.index.ntotal)

def buscar_documentos_similares_cosine(query, documentos, vectores, embedding_model, k=3):
    """
    Busca los k documentos más similares usando cosine similarity.
    """
    query_embedding = calcular_embeddings(query, embedding_model)
    similitudes = cosine_similarity(query_embedding, vectores)[0]
    top_indices = np.argsort(similitudes)[-k:][::-1]
    return [documentos[i] for i in top_indices]

def responder_consulta(query, vectordb, documentos, k=3 ): #K=3 te coge los tres documentos más relevantes por lo que si preguntas de cuantos medicamentos dispones de informacion te va a decir 3
    """
    Responde a la consulta utilizando los documentos más relevantes y cosine similarity.
    """
    # Obtener los vectores desde el índice FAISS
    vectores = obtener_vectores_faiss(vectordb)

    # Buscar los documentos más similares usando cosine similarity
    docs_relevantes = buscar_documentos_similares_cosine(query, documentos, vectores, embedding_model, k)

    medicamentos_text = "Información relevante de medicamentos:\n\n"

    for doc in docs_relevantes:
        row = doc.metadata #Cogemos la metadata (información adicional del medicamento)
        texto = (
            f"Nº Registro: {row['Nº Registro']}, Medicamento: {row['Medicamento']}, "
            f"Laboratorio: {row['Laboratorio']}, Fecha Aut.: {row['Fecha Aut.']}, Estado: {row['Estado']}, "
            f"Fecha Estado: {row['Fecha Estado']}, Cód. ATC: {row['Cód. ATC']}, Principios Activos: {row['Principios Activos']}, "
            f"Nº P. Activos: {row['Nº P. Activos']}, ¿Comercializado?: {row['¿Comercializado?']}, "
            f"¿Triangulo Amarillo?: {row['¿Triangulo Amarillo?']}, Observaciones: {row['Observaciones']}, "
            f"¿Sustituible?: {row['¿Sustituible?']}, ¿Afecta conducción?: {row['¿Afecta conducción?']}, "
            f"¿Problemas de suministro?: {row['¿Problemas de suministro?']}, Nombre: {row['Nombre']}\n"
        )
        texto += f"Sección relevante:\n{doc.page_content.strip()}\n" 
        medicamentos_text += texto + "\n"

    prompt = f"""
    Eres un asistente médico especializado en información farmacéutica. 
    Responde usando ÚNICAMENTE la siguiente información:

    {medicamentos_text}

    Instrucciones importantes:
    1. Sé preciso y conciso.
    2. Si no hay información relevante, indica que no dispones de esos datos.
    3. Nunca inventes información.
    4. Cita solo los medicamentos con información relevante.
    5. Resume la información, no la copies.
    6. No me digas en que sección se encuentra la información, solo dame la respuesta.

    Pregunta: {query}
    """

    response = llm.invoke(prompt)
    display(Markdown(response.content))


query = "Cuales son las combinaciones no deseadas del medicamento Diacomit cápsulas duras y por qué"
responder_consulta(query, vectordb, documentos)

Las combinaciones no deseadas de Diacomit cápsulas duras son:

*   Alcaloides del cornezuelo del centeno (ergotamina, dihidroergotamina): Posibilidad de ergotismo con necrosis de las extremidades.
*   Cisaprida, halofantrina, pimozida, quinidina, bepridil: Aumento del riesgo de arritmias cardiacas.
*   Inmunosupresores (tacrolimus, ciclosporina, sirolimus): Elevación de los niveles sanguíneos de inmunosupresores.
*   Estatinas (atorvastatina, simvastatina, etc.): Aumento del riesgo de reacciones adversas dependientes de la dosis, tales como rabdomiólisis.

Los documentos que cogen como relevantes tienen la siguiente estructura:

In [41]:
# Document(
#     page_content="Texto del prospecto...",
#     metadata={
#         "Nº Registro": "12345",
#         "Medicamento": "Paracetamol",
#         "Laboratorio": "Pfizer",
#         ...
#     }
# )

Información de los prospecto para verificar que responde.

In [44]:
#codigo='00131035'
codigo='06367002' #Diacomit cápsulas duras
#codigo='06367008' #Diacomit polvo para solución oral
#codigo='06366018' #Tandemact
#codigo='06356001'
#codigo='06361001'
url='https://cima.aemps.es/cima/dochtml/ft/'+codigo+'/FichaTecnica.html'
resp = requests.get(url)
print(url)

https://cima.aemps.es/cima/dochtml/ft/06367002/FichaTecnica.html


Creación de langraph para corregir faltas de ortografía y revisar la validez de la respuesta. Se crea una lista con los nombres de los medicamentos para que esos no los corriga el programa. Al ser nombres muy técnicos el programa corregía los nombres.

In [12]:

# Definimos el estado que pasará de nodo en nodo
class GraphState(TypedDict):
    query: str
    pregunta_valida: bool
    respuesta: str
    respuesta_valida: bool
    medicamentos_text: str

nombres_validos = set(nombre.split()[0].lower() for nombre in medicamentos_filtrados['Medicamento'])


def calcular_embeddings(query, embedding_model):
    """
    Calcula el embedding de la consulta.
    """
    return np.array(embedding_model.embed_query(query)).reshape(1, -1)

def obtener_vectores_faiss(vectordb):
    """
    Extrae los vectores desde el índice FAISS. 
    """
    # Accedemos a los vectores directamente desde el índice FAISS. FAISS almacena los vectores internamente.
    return vectordb.index.reconstruct_n(0, vectordb.index.ntotal)

def buscar_documentos_similares_cosine(query, documentos, vectores, embedding_model, k=3):
    """
    Busca los k documentos más similares usando cosine similarity.
    """
    query_embedding = calcular_embeddings(query, embedding_model)
    similitudes = cosine_similarity(query_embedding, vectores)[0]
    top_indices = np.argsort(similitudes)[-k:][::-1]
    return [documentos[i] for i in top_indices]

# Función 1: Validar pregunta
def validar_pregunta(state: GraphState) -> GraphState:
    pregunta = state["query"]
    tool = language_tool_python.LanguageTool('es', remote_server='https://api.languagetool.org')
    matches = tool.check(pregunta)

    # Solo filtramos errores ortográficos (no estilo o gramática)
    ortografia_errores = []
    for match in matches:
        if match.ruleIssueType == 'misspelling':
            palabra_error = pregunta[match.offset: match.offset + match.errorLength]
            if palabra_error.lower() not in nombres_validos:
                ortografia_errores.append(match)
    
    if ortografia_errores:
        pregunta_corregida = pregunta
        for error in reversed(ortografia_errores):
            if error.replacements:
                start, end = error.offset, error.offset + error.errorLength
                pregunta_corregida = pregunta_corregida[:start] + error.replacements[0] + pregunta_corregida[end:]
        
        return {
            **state,
            "pregunta_valida": False,
            "query": pregunta_corregida,
            "motivo": 'La pregunta contiene errores ortográficos y ha sido corregida.'
        }
    else:
        return {
            **state,
            "pregunta_valida": True,
            "motivo": 'La pregunta está correctamente escrita.'
        }

# Función 2: Responder consulta
def responder(state: GraphState) -> GraphState:
    query = state["query"]
    respuesta, medicamentos_text = responder_consulta_return(query, vectordb, documentos)
    return {**state, "respuesta": respuesta, "medicamentos_text": medicamentos_text}

# Función 3: Validar respuesta (mejorada con control de precisión y concisión)
def validar_respuesta(state: GraphState) -> GraphState:
    query = state["query"]
    respuesta = state["respuesta"]
    medicamentos_text = state["medicamentos_text"]

    # Evaluación de validez, concisión y resumen
    prompt = f"""
    Eres un asistente médico experto. Evalúa si esta respuesta cumple con los siguientes criterios:
    
    Criterios:
    1. Es coherente con la información proporcionada (no inventa datos).
    2. Es precisa: responde directamente y sin rodeos a lo que se pregunta.
    3. Es concisa: utiliza la menor cantidad de palabras posible sin perder claridad, y evita tecnicismos innecesarios.
    4. No es una copia literal del texto, sino un resumen adaptado a la pregunta.

    Información disponible:
    {medicamentos_text}
    
    Pregunta: {query}
    Respuesta: {respuesta}
    
    Si cumple **todos** los criterios, responde exactamente con "True".
    Si no los cumple, responde exactamente con "False".
    """
    evaluation = llm.invoke(prompt).content.strip()

    if evaluation == "True":
        return {**state, "respuesta_valida": True} #state sirve para almacenar todo el nodo anterior 
    else:
        # Corregir la respuesta para que cumpla todos los criterios
        correction_prompt = f"""
        Mejora la siguiente respuesta para que cumpla con estos criterios:
        1. Sea coherente con la información proporcionada (no inventes).
        2. Sea precisa: responde directamente a la pregunta.
        3. Sea un resumen claro, no una copia literal del texto original.

        Información disponible:
        {medicamentos_text}
        
        Pregunta: {query}
        Respuesta original: {respuesta}
        
        Devuelve únicamente la respuesta corregida, sin explicaciones adicionales.
        """
        corrected_response = llm.invoke(correction_prompt).content.strip()
        return {**state, "respuesta": corrected_response, "respuesta_valida": False}


# Adaptamos responder_consulta_return
def responder_consulta_return(query, vectordb, documentos, k=3):
    vectores = obtener_vectores_faiss(vectordb)
    docs_relevantes = buscar_documentos_similares_cosine(query, documentos, vectores, embedding_model, k)

    medicamentos_text = "Información relevante de medicamentos:\n\n"
    for doc in docs_relevantes:
        row = doc.metadata
        texto = (
            f"Nº Registro: {row['Nº Registro']}, Medicamento: {row['Medicamento']}, "
            f"Laboratorio: {row['Laboratorio']}, Fecha Aut.: {row['Fecha Aut.']}, Estado: {row['Estado']}, "
            f"Fecha Estado: {row['Fecha Estado']}, Cód. ATC: {row['Cód. ATC']}, Principios Activos: {row['Principios Activos']}, "
            f"Nº P. Activos: {row['Nº P. Activos']}, ¿Comercializado?: {row['¿Comercializado?']}, "
            f"¿Triangulo Amarillo?: {row['¿Triangulo Amarillo?']}, Observaciones: {row['Observaciones']}, "
            f"¿Sustituible?: {row['¿Sustituible?']}, ¿Afecta conducción?: {row['¿Afecta conducción?']}, "
            f"¿Problemas de suministro?: {row['¿Problemas de suministro?']}, Nombre: {row['Nombre']}\n"
        )
        texto += f"Sección relevante:\n{doc.page_content.strip()}\n"
        medicamentos_text += texto + "\n"

    prompt = f"""
    Eres un asistente médico especializado en información farmacéutica. 
    Responde usando ÚNICAMENTE la siguiente información:

    {medicamentos_text}

    Instrucciones importantes:
    1. Sé preciso y conciso.
    2. Si no hay información relevante, indica que no dispones de esos datos.
    3. Nunca inventes información.
    4. Cita solo los medicamentos con información relevante.
    5. Resume la información, no la copies.
    6. No me digas en que sección se encuentra la información, solo dame la respuesta.

    Pregunta: {query}
    """
    response = llm.invoke(prompt)
    return response.content, medicamentos_text

# Creamos el grafo
graph = StateGraph(GraphState)

# Añadimos nodos
graph.add_node("validar_pregunta", validar_pregunta)
graph.add_node("responder", responder)
graph.add_node("validar_respuesta", validar_respuesta)

# Definimos transiciones
graph.set_entry_point("validar_pregunta")
graph.add_edge("validar_pregunta", "responder")
graph.add_edge("responder", "validar_respuesta")
graph.add_edge("validar_respuesta", END)

# Compilamos el grafo
app = graph.compile()


entrada={"query":'¿Diacomit cápsulas duras contiene lactosa u otros halérgenos?'}
salida = app.invoke(entrada)

print(salida)
display(Markdown(salida['respuesta']))

{'query': '¿Diacomit cápsulas duras contiene lactosa u otros alérgenos?', 'pregunta_valida': False, 'respuesta': 'Diacomit cápsulas duras no contiene información sobre lactosa, pero sí contiene gelatina, dióxido de titanio, eritrosina, indigotina y óxido de hierro negro.', 'respuesta_valida': False, 'medicamentos_text': 'Información relevante de medicamentos:\n\nNº Registro: 06367008, Medicamento: DIACOMIT 250 mg POLVO PARA SUSPENSION ORAL, Laboratorio: Biocodex, Fecha Aut.: 12/11/2009, Estado: Autorizado, Fecha Estado: 12/11/2009, Cód. ATC: N03AX17, Principios Activos: ESTIRIPENTOL, Nº P. Activos: 1, ¿Comercializado?: SI, ¿Triangulo Amarillo?: NO, Observaciones: Diagnóstico Hospitalario, ¿Sustituible?: nan, ¿Afecta conducción?: NO, ¿Problemas de suministro?: NO, Nombre: DIACOMIT\nSección relevante:\nEncabezado: 2. COMPOSICIÓN CUALITATIVA Y CUANTITATIVA\nContenido: Cada sobre contiene 250\nmg de estiripentol\nExcipiente con efecto conocido\n0,11\nmg de sodio por sobre.\nCada sobre cont

Diacomit cápsulas duras no contiene información sobre lactosa, pero sí contiene gelatina, dióxido de titanio, eritrosina, indigotina y óxido de hierro negro.